## Comparando registros no CouchDB com ES

In [12]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import couchdb
import json
import copy

In [27]:
db_name = 'endemicas_rio_de_janeiro'
#db_name = 'teste'
filename = 'dbs/' + db_name + '.json'
with open(filename) as data_file:    
    data_original = json.load(data_file)

In [48]:
data = copy.deepcopy(data_original)

In [15]:
print "Retrieved %d records from CouchDB" % len(data['rows'])

Retrieved 22559 records from CouchDB


####Verificando se todos os registros possuem um identificador único

In [16]:
ids = [record['id'] for record in data['rows']]
print len(set(ids))

22559


In [18]:
# Set ES connection
es = Elasticsearch('http://10.10.100.48:32768/')
es.ping()

True

In [46]:
es.indices.delete(index='endemicas_rio_de_janeiro', ignore=[400, 404])

{u'acknowledged': True}

In [47]:
es.indices.create(index='endemicas_rio_de_janeiro')

{u'acknowledged': True}

In [49]:
data['rows'][0]

{u'doc': {u'_id': u' HABITAT: Ep\xedfita em Palmeiras',
  u'_rev': u'9-49f4ee395432283aff6631e3d8d31444',
  u'acceptedNameUsage': u' DESC: Herb\xe1ceo ep\xedfita',
  u'collectionCode': u'HPAN',
  u'comments': u'occurrence:rj:3432165',
  u'coordinateUncertaintyInMeters': u'50 a 100 km',
  u'country': u'Brasil',
  u'dateIdentified': u'2003-11-',
  u'decimalLatitude': -16.557583,
  u'decimalLongitude': -57.806658,
  u'family': u'Araceae',
  u'genus': u'Philodendron',
  u'georeferenceProtocol': u'google earth',
  u'georeferenceVerificationStatus': u'ok',
  u'id': u' HABITAT: Ep\xedfita em Palmeiras',
  u'identifiedBy': u'Carniello, M.A.',
  u'institutionCode': u'UNEMAT',
  u'locality': u'Porto Lim\xe3o',
  u'metadata': {u'contact': u'fernanda@cncflora.net ; lucasmoraes@cncflora.net',
   u'contributor': u'Fernanda Wimmer ; Lucas Moraes',
   u'created': 1426265349,
   u'creator': u'Lucas Moraes',
   u'modified': 1426977275,
   u'type': u'occurrence'},
  u'modified': u'2014-08-20 18:02:58-03'

In [50]:
not_inserted = []
inserted = []
for doc in data['rows']:
    doc['doc']['id'] = doc['doc']['_id']
    doc['doc']['rev'] = doc['doc']['_rev']
    del doc['doc']['_id']
    del doc['doc']['rev']
    doc_type = doc['doc']['metadata']['type']
    record_before = es.count(index=db_name, doc_type=doc_type)
    es.index(index=db_name, doc_type=doc_type, body=doc['doc'], id=doc['doc']['id'])
    es.indices.refresh(index=db_name)
    record_after = es.count(index=db_name, doc_type=doc_type)
    try:
        assert record_before['count'] == record_after['count']-1
        inserted.append('%s' % doc['doc']['id'])
    except AssertionError:
        #"Document %s was not inserted" % doc['doc']['id']
        not_inserted.append('%s' % doc['doc']['id'])

In [38]:
len(inserted)

22559